# Import All we Need

In [ ]:
# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# keras imports 
from keras.applications.densenet import DenseNet169 , preprocess_input


from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import GlobalAveragePooling2D

# other imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from __future__ import print_function


from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics

import matplotlib.pyplot as plt
import numpy as np
from numpy import mean
from numpy import std
import seaborn as sns

import pickle
import glob
import cv2
import h5py
import os
import json



# load the user configs , from config.json


In [ ]:
# load the user configs

# use your own path
with open('/content/drive/MyDrive/covid19/config/config.json') as f:    
  config = json.load(f)


# config variables
model_name      = config["model"]
weights         = config["weights"]
include_top     = config["include_top"]
train_path      = config["train_path"]
features_path   = config["features_path"]
labels_path     = config["labels_path"]
test_size       = config["test_size"]
model_path      = config["model_path"]



# Use DenseNet169 as Model

In [ ]:

# DenseNet169 as model 

if model_name == "DenseNet169":
  base_model = DenseNet169(include_top=include_top, weights=weights, input_tensor=Input(shape=(224,224,3)), input_shape=(224,224,3), pooling="avg")
  model = Model(base_model.input, base_model.get_layer('avg_pool').output)
  image_size = (224, 224)

else:
  base_mode = none


base_model.summary()

# encoding labels

In [ ]:
# path to training dataset
train_labels = os.listdir(train_path)

# encode the labels
print("[INFO] encoding labels...")
le = LabelEncoder()
le.fit([tl for tl in train_labels])



# Feature Extraction


In [ ]:
# variables to hold features and labels
features = []
labels   = []


# loop over all the labels in the folder
count = 1
for i, label in enumerate(train_labels):
  cur_path = train_path + "/" + label
  count = 1
  for image_path in glob.glob(cur_path + "/*"):
    img = image.load_img(image_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    features.append(flat)
    labels.append(label)
    print("[INFO] processed - " + str(count))
    count += 1
  print("[INFO] completed label - " + label)

# encode the labels using LabelEncoder
le = LabelEncoder()
le_labels = le.fit_transform(labels)

# get the shape of  training labels
print("[STATUS] training labels: {}".format(le_labels))
print("[STATUS] training labels shape: {}".format(le_labels.shape))



# save features and labels

In [ ]:
# save features and labels
h5f_data = h5py.File(features_path, 'w')
h5f_data.create_dataset('dataset_1', data=np.array(features))

h5f_label = h5py.File(labels_path, 'w')
h5f_label.create_dataset('dataset_1', data=np.array(le_labels))

h5f_data.close()
h5f_label.close()

print("[STATUS] features and labels saved..")

[STATUS] features and labels saved..


# save model and weights


In [ ]:
# save model and weights
model_json = model.to_json()
with open(model_path + str(test_size) + ".json", "w+") as json_file:
  json_file.write(model_json)

# save weights
model.save_weights(model_path + str(test_size) + ".h5")

print("[STATUS] saved model and weights to disk..")

[STATUS] saved model and weights to disk..


# load the user configs


In [ ]:

# load the user configs
with open('/content/drive/MyDrive/covid19/config/config.json') as f:    
  config = json.load(f)

# config variables
test_size       = config["test_size"]
seed            = config["seed"]
features_path   = config["features_path"]
labels_path     = config["labels_path"]
results         = config["results"]
classifier_path = config["classifier_path"]
train_path      = config["train_path"]
num_classes     = config["num_classes"]


# import features and labels


In [ ]:
# import features and labels
h5f_data  = h5py.File(features_path, 'r')
h5f_label = h5py.File(labels_path, 'r')

features_string = h5f_data['dataset_1']
labels_string   = h5f_label['dataset_1']

features = np.array(features_string)
labels   = np.array(labels_string)


h5f_data.close()
h5f_label.close()

# features Selection

In [ ]:

# Select features according to the k highest scores.
n = int(features.shape[1]*7/100)
fs = SelectKBest(score_func=f_classif, k=n)
X_selected = fs.fit_transform(features, labels)

# verify the shape of features and labels
print("[INFO] features shape: {}".format(X_selected.shape))
print("[INFO] labels shape: {}".format(labels.shape))



[INFO] features shape: (1125, 116)
[INFO] labels shape: (1125,)


# Splitting features into train and test set

In [ ]:
# split the training and testing data
print("[INFO] training started...")
(XtrainData, XtestData, XtrainLabels, XtestLabels) = train_test_split(np.array(X_selected), np.array(labels),test_size=test_size,random_state=670)


[INFO] training started...


# use LighGBM Classification as the model

In [ ]:
# use LighGBM Classification as the model
model = LGBMClassifier(max_depth= 3, learning_rate=0.24,n_estimators= 200)

# fit and predict model

In [ ]:
# fit the model
model.fit(XtrainData, XtrainLabels)

# predict model
preds = model.predict(XtestData)

# confusion matrix

In [ ]:

print("[INFO] Generate confusion matrix")

# confusion_matrix
cm = confusion_matrix(XtestLabels, preds)


fig, ax = plt.subplots()
sns.set(font_scale=1.4) # Adjust to fit

labels = sorted(list(os.listdir(train_path)))
sns.heatmap(cm,
            yticklabels = labels,
            xticklabels = labels,
            annot=True,
            cmap='binary'
            )

ax.tick_params(axis='both', which='major', labelsize=12)  # Adjust to fit


saved confusion matrix 

In [ ]:
print("[INFO] saved confusion matrix as svg to disk..")

image_format = 'svg' # e.g .png, .svg, etc.
image_name = '/content/drive/MyDrive/covid19/output/3class_matrix.svg'

fig.savefig(image_name, format=image_format, dpi=1200)


plt.show()


# Report

In [ ]:
result = {}

Y_test = XtestLabels
Y_pred = preds

result["Accuracy score"] =  metrics.accuracy_score(Y_test, Y_pred)
result["confusion matrix"] = metrics.confusion_matrix(Y_test, Y_pred)
result["f1 score"] = metrics.f1_score(Y_test, Y_pred, average="macro")
result["precision score"] = metrics.precision_score(Y_test, Y_pred, average="macro")
result["Sensitivity"] = result["confusion matrix"][0,0] / (result["confusion matrix"][0,0] + result["confusion matrix"][0,1])
result["Specifity"] = result["confusion matrix"][1,1] / (result["confusion matrix"][1,0] + result["confusion matrix"][1,1])

print(f"\nAccuracy score :  %.4f" % result["Accuracy score"])
print(f"\nconfusion matrix : \n" , result["confusion matrix"])
print(f"\nf1 score :  %.4f" % result["f1 score"])
print(f"\nprecision score :  %.4f" % result["precision score"])
print(f"\nSpecifity :  %.4f" % result["Specifity"])
print(f"\nSensitivity :  %.4f" % result["Sensitivity"])
